# Custom Question Answering

https://learn.microsoft.com/en-us/azure/ai-services/language-service/question-answering/overview?wt.mc_id=MVP_322781

## Install Library

In [ ]:
%pip install azure-ai-language-questionanswering

## Load Azure Configurations

In [ ]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

## Create a Question Answering Client

In [ ]:
from azure.ai.language.questionanswering import QuestionAnsweringClient
from azure.ai.language.questionanswering.models import ShortAnswerOptions
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticate the Question Answering client using Azure Key Credential.

    Returns:
        QuestionAnsweringClient: Authenticated client for Question Answering.
    """
    qac_credential = AzureKeyCredential(language_key)
    question_answering_client = QuestionAnsweringClient(
        language_endpoint, 
        qac_credential)
    
    return question_answering_client

# Initialize the client
client = authenticate_client()

## Scenario 1: With a Project in the Language Studio

Gets data from a file: https://download.microsoft.com/download/7/B/1/7B10C82E-F520-4080-8516-5CF0D803EEE0/surface-book-user-guide-EN.pdf

In [ ]:
# The name of your question answering project in Language Studio
knowledge_base_project = "Sample-project-01"

# There are two possible values: test, and production. 
# production is dependent on you having deployed your project from Language Studio > question answering > Deploy project.
deployment = "production"

## Get Answers function

In [51]:
def get_answers_with_project(client, question, knowledge_base_project, deployment):
    """
    Retrieves answers from the Azure Custom Question Answering service for a given question.

    Args:
        client (QuestionAnsweringClient): The authenticated Question Answering client.
        question (str): The question to query the knowledge base.
        knowledge_base_project (str): The name of the knowledge base project in Azure Language Studio.
        deployment (str): The deployment environment (e.g., 'test' or 'production').

    Returns:
        None: Prints the answers and their details to the console.
    """
    output = client.get_answers(
        project_name=knowledge_base_project,
        deployment_name=deployment,
        question=question,
        confidence_threshold=0.65,
        top=3,
        short_answer_options=ShortAnswerOptions(confidence_threshold=0.7, top=1),
    )

    print(f"Q: {question}")
    for idx, answer in enumerate(output.answers):
        print(f"Answer {idx + 1}:")
        print(f"Alternate questions: {answer.questions}")
        print(f"Answer Text: {answer.answer}")
        print(f"Confidence: {answer.confidence:.2f}")
        print(f"Source: {answer.source}")

        # Check if a short answer is available and print its details
        if answer.short_answer:
            short_answer = answer.short_answer
            print("Short Answer:")
            print(f"  Text: {short_answer.text.strip()}")
            print(f"  Confidence: {short_answer.confidence:.2f}")
            print(f"  Offset: {short_answer.offset}")
            print(f"  Length: {short_answer.length}")

        # Display follow-up prompts if available
        if answer.dialog and answer.dialog.prompts:
            print("Follow-up Prompts:")
            for prompt in answer.dialog.prompts:
                print(f"  - {prompt.display_text}")  # Display text of the follow-up prompt
        print("-" * 50)  # Separator for readability

In [53]:
question = "How do I setup my surface book?"
get_answers_with_project(client, question, knowledge_base_project, deployment)

Q: How do I setup my surface book?
Answer 1:
Alternate questions: ['Set up your Surface Book', 'What are the steps to set up a new Surface Book?', 'How do I configure my Surface Book for the first time?']
Answer Text: **Set up your Surface Book**

Before you start, make sure a wireless network is available and you have the network password.

If you have a  [Microsoft account,](http://windows.microsoft.com/en-us/windows-10/getstarted-microsoft-account)  use it to set up your Surface Book. A Microsoft account is an email address and password you use to sign in to other Microsoft services like Xbox, Outlook.com, OneDrive, Skype, or Windows Phone. If you don’t already have a Microsoft account, you can create one during setup.
Confidence: 0.91
Source: https://download.microsoft.com/download/7/B/1/7B10C82E-F520-4080-8516-5CF0D803EEE0/surface-book-user-guide-EN.pdf
Follow-up Prompts:
  - Charge your Surface Book
  - Set up Windows
  - Choose a mode to work in
---------------------------------

## Scenario 2: Query text documents without a project

You do not have a project but you have all the documents in text.

In [30]:
text_documents = [
    """
    Surface Book Features
    Power button: Press the power button to turn your Surface Book on. You can also use the power button to put your Surface Book to sleep and wake it when you’re ready to start working again. 
    Clipboard: Detach the thin, light Clipboard from the keyboard for mobile computing and easy presentation in tablet mode. 
    Use the 13.5” full HD display, with its 3:2 aspect ratio and 3000 x 2000 resolution display, to watch HD movies, browse the web, and use your favorite apps.
    """,
    """
    Set up your Surface Book
    Before you start, make sure a wireless network is available and you have the network password. 
    If you have a Microsoft account, use it to set up your Surface Book. 
    A Microsoft account is an email address and password you use to sign in to other Microsoft services like Xbox, Outlook.com, OneDrive, Skype, or Windows Phone. 
    If you don’t already have a Microsoft account, you can create one during setup.
    """,
    """
    Charge your Surface Book
    1. Connect the two parts of the power cord.
    2. Connect the power cord securely to the charging port on your Surface Book. A light appears at the end of the connector when your Surface is getting power.
    3. Plug the power supply into an electrical outlet.
    """,
    """
    Power and charging
    Surface Book has two batteries—one in the keyboard and one in the Clipboard. 
    It takes two to four hours to charge Surface Book fully from an empty state. 
    It can take longer if you’re using your Surface Book for power-intensive activities like gaming or video streaming while you’re charging it. 
    You can use the USB port on your power supply to charge other devices, like a phone, while your Surface Book charges.
    """,
    """
    Check the battery level
    You can check the battery level from the lock screen or on the desktop.
    Lock screen: When you wake your Surface Book, the battery status appears in the lower-right corner of the lock screen.
    Desktop taskbar: On the desktop, select the battery status icon on the right side of the taskbar. 
    The battery status pane shows the charging level for both batteries. When your Surface Book is fully charged, it should last up to 10 hours for web browsing.
    """,
]

## Get Answer From Text Function

In [39]:
from azure.ai.language.questionanswering import models as qna

def get_answers_no_project(client, question, text_documents):
    """
    Retrieves answers from the Azure Custom Question Answering service for a given question
    using a set of text documents instead of a pre-defined project.

    Args:
        client (QuestionAnsweringClient): The authenticated Question Answering client.
        question (str): The question to query the text documents.
        text_documents (list of str): A list of text documents to search for answers.

    Returns:
        None: Prints the answers and their details to the console.
    """
    # Create an input object for the Question Answering service
    # This specifies the question and the text documents to search
    input = qna.AnswersFromTextOptions(
        question=question,
        text_documents=text_documents
    )

    # Call the Question Answering service to get answers from the text documents
    output = client.get_answers_from_text(input)

    # Print the question being asked
    print(f"Q: {question}")
    
    # Iterate through the answers returned by the service
    for idx, answer in enumerate(output.answers):
        # Print the answer details
        print(f"Answer {idx + 1}:")
        print(f"Answer Text: {answer.answer}")  # Full answer text
        print(f"Confidence: {answer.confidence:.2f}")  # Confidence score

In [40]:
question = "How many inches is the display?"
get_answers_no_project(client, question, text_documents)

Q: How many inches is the display?
Answer 1:
Answer Text: Use the 13.5” full HD display, with its 3:2 aspect ratio and 3000 x 2000 resolution display, to watch HD movies, browse the web, and use your favorite apps.
Confidence: 0.58
Answer 2:
Answer Text: You can also use the power button to put your Surface Book to sleep and wake it when you’re ready to start working again. 
    Clipboard: Detach the thin, light Clipboard from the keyboard for mobile computing and easy presentation in tablet mode. 
    Use the 13.5” full HD display, with its 3:2 aspect ratio and 3000 x 2000 resolution display, to watch HD movies, browse the web, and use your favorite apps.
Confidence: 0.48
Answer 3:
Answer Text: Clipboard: Detach the thin, light Clipboard from the keyboard for mobile computing and easy presentation in tablet mode. 
    Use the 13.5” full HD display, with its 3:2 aspect ratio and 3000 x 2000 resolution display, to watch HD movies, browse the web, and use your favorite apps.
Confidence: 